## Login to OpenShift with API Token

First, let's login to OpenShift cluster using `oc` tool. 

In the Red Hat OpenShift console, click on your username and select `Copy login command`.

![copy-login.png](notebook-files/copy-login.png)

Click on `Display Token` and your API token will appear.

![log-in-with-token.png](notebook-files/log-in-with-token.png)

Copy `Log in with token` command and paste it in the cell below. The command has your `<user-API-token>` and `<cluster-DNS-name>`.

In [ ]:
!oc login --token=<user-API-token> --server=https://api.<cluster-DNS-name>:6443

Create `ovms-operator` project and to go this project.

In [3]:
!oc new-project ovms-operator 
!oc project ovms-operator

Error from server (AlreadyExists): project.project.openshift.io "ovms-operator" already exists
Already on project "ovms-operator" on server "https://api.openvino5.3q12.p1.openshiftapps.com:6443".


## Create an OVMS Instance


Here's the yaml used to configure the OVMS service. We specified `name` to be `ovms-resnet` and `model_path` to be `gs://ovms-public-eu/resnet50-binary`. Also, we defined `model_name` here; we will use this value for gRPC and REST API calls.

In [94]:
!cat ovms.yaml

apiVersion: intel.com/v1alpha1
kind: Ovms
metadata:
  name: ovms-resnet
spec:
  aws_access_key_id: ""
  aws_region: ""
  aws_secret_access_key: ""
  grpc_port: 8080
  image_name: registry.connect.redhat.com/intel/openvino-model-server:latest
  log_level: INFO
  model_name: "resnet"
  model_path: "gs://ovms-public-eu/resnet50-binary"
  plugin_config: '{\"CPU_THROUGHPUT_STREAMS\":\"1\"}'
  replicas: 1
  resources:
    limits:
      cpu: 4
      memory: 500Mi
  rest_port: 8081
  service_type: ClusterIP

Run the cell below to create new OVMS service called `ovms-resnet`.

In [95]:
!oc apply -f ovms.yaml

ovms.intel.com/ovms-resnet created


Let's see if pod and service were created. They should start with `ovms-resnet`.

In [99]:
!oc get pod
!oc get service

NAME                             READY     STATUS    RESTARTS   AGE
minio-5c57f888dd-wlv4m           1/1       Running   0          25h
ovms-instance-795cf867f8-42b5z   1/1       Running   0          2d19h
ovms-resnet-5fd9cd8b86-6nmv4     1/1       Running   0          107s
NAME            TYPE        CLUSTER-IP      EXTERNAL-IP   PORT(S)             AGE
minio-service   ClusterIP   172.30.53.134   <none>        9000/TCP            25h
ovms-instance   ClusterIP   172.30.187.42   <none>        8080/TCP,8081/TCP   7d19h
ovms-resnet     ClusterIP   172.30.91.166   <none>        8080/TCP,8081/TCP   107s


Check if the service is up and running by making a REST API call.

In [131]:
!curl http://ovms-resnet.ovms-operator.svc.cluster.local:8081/v1/models/resnet

{
 "model_version_status": [
  {
   "version": "1",
   "state": "AVAILABLE",
   "status": {
    "error_code": "OK",
    "error_message": "OK"
   }
  }
 ]
}


The `state` parameter should be `AVAILABLE`. Note `version` value, we will use it for gRPC and REST API calls. 

## gRPC API Calls

In this section, we will make gRPC API calls to the OVMS service using sample scripts.

To run sample scripts, we will need to get the serving metadata. `get_serving_meta.py` script has the following arguments:
* **--grpc_address**  
* **--grpc_port**
* **--model_name**
* **--model_version**

Run the cell to get the metadata.

In [133]:
!python get_serving_meta.py --grpc_address ovms-resnet.ovms-operator.svc.cluster.local \
                            --grpc_port 8080 \
                            --model_name resnet \
                            --model_version 1

2021-04-23 20:06:23.152381: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-04-23 20:06:23.152419: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Getting model metadata for model: resnet
Inputs metadata:
	Input name: 0; shape: [1, 3, 224, 224]; dtype: DT_FLOAT
Outputs metadata:
	Output name: 1463; shape: [1, 1000]; dtype: DT_FLOAT


For the `jpeg_classificaion.py` script, we specified `--input_name` and `--output_name` arguments, from the metadata output. However, we omitted the `--images_list` argument because we used the default value, `input_images.txt`. 

Let's see what's in that file.

In [136]:
!cat input_images.txt

    images/airliner.jpeg 404
    images/arctic-fox.jpeg 279
    images/bee.jpeg 309
    images/golden_retriever.jpeg 207
    images/gorilla.jpeg 366
    images/magnetic_compass.jpeg 635
    images/peacock.jpeg 84
    images/pelican.jpeg 144
    images/snail.jpeg 113
    images/zebra.jpeg 340


As you can see, `input_images.txt` contains paths to images and [class ids](classes.py).

Run a classification inference on list of images, listed in `input_images.txt`.

In [5]:
!python3 jpeg_classification.py --grpc_address ovms-resnet.ovms-operator.svc.cluster.local \
                                --grpc_port 8080 \
                                --input_name 0 \
                                --output_name 1463

2021-04-23 22:40:47.370032: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-04-23 22:40:47.370069: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Start processing:
	Model name: resnet
	Images list file: input_images.txt
images/airliner.jpeg (1, 3, 224, 224) ; data range: 0.0 : 255.0
Processing time: 15.37 ms; speed 65.07 fps
	 1 airliner 404 ; Correct match.
images/arctic-fox.jpeg (1, 3, 224, 224) ; data range: 0.0 : 255.0
Processing time: 7.11 ms; speed 140.73 fps
	 2 Arctic fox, white fox, Alopex lagopus 279 ; Correct match.
images/bee.jpeg (1, 3, 224, 224) ; data range: 0.0 : 255.0
Processing time: 7.10 ms; speed 140.88 fps
	 3 bee 309 ; Correct match.
images/golden_retriever.jpeg (1, 3, 224, 224) ; data range: 0.0 : 255.0
Processing time: 6.77 

Here's a different image list you could try, `people_images_list.txt`. 

In [137]:
!cat people_images_list.txt

images/people/people1.jpeg 1
images/people/people2.jpeg 1
images/people/people3.jpeg 1
images/people/people4.jpeg 1

This time, we will run a classification inference on people images by specifying `--images_list`.

In [8]:
!python3 jpeg_classification.py --grpc_address ovms-resnet.ovms-operator.svc.cluster.local \
                                --grpc_port 8080 \
                                --input_name 0 \
                                --output_name 1463 \
                                --images_list people_images_list.txt

2021-04-23 22:49:28.461951: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-04-23 22:49:28.461987: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Start processing:
	Model name: resnet
	Images list file: people_images_list.txt
images/people/people1.jpeg (1, 3, 224, 224) ; data range: 0.0 : 255.0
Processing time: 13.68 ms; speed 73.12 fps
Traceback (most recent call last):
  File "jpeg_classification.py", line 118, in <module>
    mark_message = "; Incorrect match. Should be {} {}".format(lb[i], classes.imagenet_classes[lb[i]] )
NameError: name 'lb' is not defined


Let's try another script, `grpc_serving_client.py`. It accepts NumPy array as an input for `--images_numpy_path` argument.

In [7]:
!python3 grpc_serving_client.py \
        --grpc_address ovms-resnet.ovms-operator.svc.cluster.local \
        --grpc_port 8080 \
        --input_name 0 \
        --output_name 1463 \
        --images_numpy_path imgs.npy \
        --transpose_input False

2021-04-23 22:48:48.174710: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-04-23 22:48:48.174749: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Image data range: 0.0 : 255.0
Start processing:
	Model name: resnet
	Iterations: 10
	Images numpy path: imgs.npy
	Images in shape: (10, 3, 224, 224)

Iteration 1; Processing time: 14.43 ms; speed 69.32 fps
imagenet top results in a single batch:
	 0 airliner 404 
Iteration 2; Processing time: 6.95 ms; speed 143.86 fps
imagenet top results in a single batch:
	 0 Arctic fox, white fox, Alopex lagopus 279 
Iteration 3; Processing time: 6.78 ms; speed 147.49 fps
imagenet top results in a single batch:
	 0 bee 309 
Iteration 4; Processing time: 6.93 ms; speed 144.34 fps
imagenet top results in a single batch:


## REST API Calls

In this section, we will make REST API calls to the OVMS service using sample scripts. We will use port `8081` as `--rest_port`, as we specified in the `ovms.yaml` when we created the OVMS service.

Run the cell below to get the model status.

In [138]:
!python rest_get_model_status.py --rest_url http://ovms-resnet.ovms-operator.svc.cluster.local \
                                 --rest_port 8081 \
                                 --model_name resnet

{
 "model_version_status": [
  {
   "version": "1",
   "state": "AVAILABLE",
   "status": {
    "error_code": "OK",
    "error_message": "OK"
   }
  }
 ]
}



Run the cell below to get serving metadata.

In [140]:
!python rest_get_serving_meta.py --rest_url http://ovms-resnet.ovms-operator.svc.cluster.local \
                                 --rest_port 8081 \
                                 --model_name resnet

{
 "modelSpec": {
  "name": "resnet",
  "signatureName": "",
  "version": "1"
 },
 "metadata": {
  "signature_def": {
   "@type": "type.googleapis.com/tensorflow.serving.SignatureDefMap",
   "signatureDef": {
    "serving_default": {
     "inputs": {
      "0": {
       "dtype": "DT_FLOAT",
       "tensorShape": {
        "dim": [
         {
          "size": "1",
          "name": ""
         },
         {
          "size": "3",
          "name": ""
         },
         {
          "size": "224",
          "name": ""
         },
         {
          "size": "224",
          "name": ""
         }
        ],
        "unknownRank": false
       },
       "name": "0"
      }
     },
     "outputs": {
      "1463": {
       "dtype": "DT_FLOAT",
       "tensorShape": {
        "dim": [
         {
          "size": "1",
          "name": ""
         },
         {
          "size": "1000",
          "name": ""
         }
        ],
        "unknownRank": false
       },
       "name": "1463"


Let's run `rest_serving_client.py` script. It takes `--images_numpy_path` and `--labels_numpy_path` as an input. The script return the same output as `grpc_serving_client.py`.

In [129]:
!python rest_serving_client.py --rest_url http://ovms-resnet.ovms-operator.svc.cluster.local \
                               --rest_port 8081 \
                               --model_name resnet \
                               --input_name 0 \
                               --output_name 1463 \
                               --images_numpy_path imgs.npy \
                               --labels_numpy_path lbs.npy \
                               --transpose_input False                

Image data range: 0 : 255
Start processing:
	Model name: resnet
	Iterations: 10
	Images numpy path: imgs.npy
	Images in shape: (10, 3, 224, 224)

output shape: (1, 1000)
Iteration 1; Processing time: 38.24 ms; speed 26.15 fps
imagenet top results in a single batch:
	 0 airliner 404 ; Correct match.
output shape: (1, 1000)
Iteration 2; Processing time: 14.91 ms; speed 67.07 fps
imagenet top results in a single batch:
	 0 Arctic fox, white fox, Alopex lagopus 279 ; Correct match.
output shape: (1, 1000)
Iteration 3; Processing time: 14.76 ms; speed 67.77 fps
imagenet top results in a single batch:
	 0 bee 309 ; Correct match.
output shape: (1, 1000)
Iteration 4; Processing time: 15.90 ms; speed 62.89 fps
imagenet top results in a single batch:
	 0 golden retriever 207 ; Correct match.
output shape: (1, 1000)
Iteration 5; Processing time: 15.30 ms; speed 65.35 fps
imagenet top results in a single batch:
	 0 gorilla, Gorilla gorilla 366 ; Correct match.
output shape: (1, 1000)
Iteration 6;